In [18]:
import pandas as pd
import json
import re

In [19]:
team_info = pd.read_csv('./data/players_21.csv', sep=',',encoding='utf-8')
team_info = team_info[team_info.long_name.notnull()]
team_info = team_info[team_info.long_name != '']

In [20]:
def remove_special_characters(input_str):
    return re.sub(r'[^a-zA-Z0-9\s]', '', input_str)

# leggi il csv e impora i dati in un dataframe in utf-8
df = pd.read_csv('./data/2021-2022-player-stats.csv', sep=';',encoding='ISO-8859-1')
df["Player"] = df["Player"].str.replace('?', '')
df["Player"] = df["Player"].apply(remove_special_characters)

C:\Users\georg\AppData\Local\Temp\ipykernel_9084\679700640.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df["Player"] = df["Player"].str.replace('?', '')


In [21]:
# example clarifications

text = "Paulo Bruno Exequiel Dybala"
substrings = text.split()

if "Paulo" in substrings and "Dybala" in substrings:
    print(True)
else:
    print(False)


True


In [22]:
team_info['long_name_cleaned'] = team_info['long_name'].apply(remove_special_characters)    
team_info.head()

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,...,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url,long_name_cleaned
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,"RW, ST, CF",93,93,103500000.0,560000.0,33,...,52+3,52+3,62+3,19+3,https://cdn.sofifa.net/players/158/023/21_120.png,https://cdn.sofifa.net/teams/241/60.png,https://cdn.sofifa.net/flags/es.png,https://cdn.sofifa.net/teams/1369/60.png,https://cdn.sofifa.net/flags/ar.png,Lionel Andrs Messi Cuccittini
1,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"ST, LW",92,92,63000000.0,220000.0,35,...,54+3,54+3,61+3,20+3,https://cdn.sofifa.net/players/020/801/21_120.png,https://cdn.sofifa.net/teams/45/60.png,https://cdn.sofifa.net/flags/it.png,https://cdn.sofifa.net/teams/1354/60.png,https://cdn.sofifa.net/flags/pt.png,Cristiano Ronaldo dos Santos Aveiro
2,188545,https://sofifa.com/player/188545/robert-lewand...,R. Lewandowski,Robert Lewandowski,ST,91,91,111000000.0,240000.0,31,...,60+3,60+3,61+3,19+3,https://cdn.sofifa.net/players/188/545/21_120.png,https://cdn.sofifa.net/teams/21/60.png,https://cdn.sofifa.net/flags/de.png,NaN,https://cdn.sofifa.net/flags/pl.png,Robert Lewandowski
3,190871,https://sofifa.com/player/190871/neymar-da-sil...,Neymar Jr,Neymar da Silva Santos Júnior,"LW, CAM",91,91,132000000.0,270000.0,28,...,49+3,49+3,62+3,20+3,https://cdn.sofifa.net/players/190/871/21_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,NaN,https://cdn.sofifa.net/flags/br.png,Neymar da Silva Santos Jnior
4,192985,https://sofifa.com/player/192985/kevin-de-bruy...,K. De Bruyne,Kevin De Bruyne,"CAM, CM",91,91,129000000.0,370000.0,29,...,69+3,69+3,75+3,21+3,https://cdn.sofifa.net/players/192/985/21_120.png,https://cdn.sofifa.net/teams/10/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1325/60.png,https://cdn.sofifa.net/flags/be.png,Kevin De Bruyne


In [23]:
def find_matching_long_name(player_name):
    substrings = player_name.split()

    def checkEquivalence(row):
        return all(substring in row for substring in substrings)
    
    matching_rows = team_info[team_info['long_name_cleaned'].apply(checkEquivalence)]

    if not matching_rows.empty:
        return matching_rows['short_name'].iloc[0]
    else:
        return player_name
    
df['Player'] = df['Player'].apply(lambda x: find_matching_long_name(x))

In [24]:
unique_club_names1 = team_info["club_name"].unique()

unique_club_names2 = df["Squad"].unique()


print(len(unique_club_names1))
print(len(unique_club_names2))

# other problem: club names are different

df = df.merge(team_info[['short_name', 'club_name']],
              left_on='Player', right_on='short_name', how='left')
df = df.drop('short_name', axis=1)
df = df.drop('Squad', axis=1)
df.rename(columns={'club_name': 'Squad'}, inplace=True)
# problem solved

unique_club_names1 = team_info["club_name"].unique()

unique_club_names2 = df["Squad"].unique()


print(len(unique_club_names1))
print(len(unique_club_names2))


682
98
682
383


In [25]:
df = df.merge(team_info[["short_name", "player_face_url", "club_logo_url","club_name"]],
              left_on=['Player', 'Squad'],
              right_on=['short_name', 'club_name'],
              how='left')

df.drop(columns=['short_name', 'club_name'], inplace=True)
df.rename(columns={"player_face_url": "PlayerFaceUrl", "club_logo_url": "ClubLogoUrl"}, inplace=True)

In [26]:
# count number of player with the same SquadName
# note: the number of player with the same SquadName is > 1 (USEFUL for ffill)
print(df['Squad'].value_counts()[df['Squad'].value_counts() < 2])

Club Deportivo Jorge Wilstermann    1
Hebei FC                            1
Al Ahli                             1
1. FC Heidenheim 1846               1
SC Heerenveen                       1
                                   ..
Paris FC                            1
Gazişehir Gaziantep F.K.            1
KAS Eupen                           1
Sydney FC                           1
Western Sydney Wanderers            1
Name: Squad, Length: 151, dtype: int64


In [27]:
df['PlayerFaceUrl'].fillna('https://cdn.sofifa.net/player_0.svg', inplace=True)
# sort by SquadName
df.sort_values(by=['Squad'], inplace=True)
# fill none values on ClubLogoUrl field with the row with the same Squad
df['ClubLogoUrl'].fillna(method='ffill', inplace=True)
# correct: no results == no Squad with only 1 player

In [28]:
# print a Player that includes González in the Player
print(df[df['Player'].str.contains('González')])

        Rk          Player Nation Pos     Comp   Age  Born  MP  Starts   Min  \
1114  1035     G. González    URU  DF  La Liga  27.0  1994   9       3   365   
1119  1036     N. González    ESP  MF  La Liga  20.0  2002  27      12  1114   
1122  1037     N. González    ARG  FW  Serie A  24.0  1998  33      26  2357   
1121  1037     N. González    ARG  FW  Serie A  24.0  1998  33      26  2357   
1118  1036     N. González    ESP  MF  La Liga  20.0  2002  27      12  1114   
1115  1035     G. González    URU  DF  La Liga  27.0  1994   9       3   365   
1111  1033  Diego González    ESP  DF  La Liga  27.0  1995  28      19  1793   
1116  1035     G. González    URU  DF  La Liga  27.0  1994   9       3   365   
1117  1036     N. González    ESP  MF  La Liga  20.0  2002  27      12  1114   
1120  1037     N. González    ARG  FW  Serie A  24.0  1998  33      26  2357   

      ...  PKwon  PKcon   OG  Recov  AerWon  AerLost  AerWon%  \
1114  ...   0.00   0.00  0.0   8.54    2.44     2.93  

In [29]:
acronyms = {"Rk": "Rank", "Player": "Player's name", "Nation": "Player's nation", "Pos": "Position",
            "Squad": "Squad’s name", "Comp": "League that squat occupies", "Age": "Player's age",
            "Born": "Year of birth", "MP": "Matches played", "Starts": "Matches started",
            "Min": "Minutes played", "90s": "Minutes played divided by 90",
            "Goals": "Goals scored or allowed",
            "Shots": "Shots total (Does not include penalty kicks)",
            "SoT": "Shots on target (Does not include penalty kicks)",
            'SoT%': 'Shots on target percentage (Does not include penalty kicks)',
            'G/Sh': 'Goals per shot',
            'G/SoT': 'Goals per shot on target (Does not include penalty kicks)',
            'ShoDist': 'Average distance, in yards, from goal of all shots taken (Does not include penalty kicks)',
            'ShoFK': 'Shots from free kicks',
            'ShoPK': 'Penalty kicks made',
            "PasProg": "Completed passes that move the ball towards the opponent's goal at least 10 yards from its furthest point in the last six passes, or any completed pass into the penalty area",
                  "PasAtt": "Passes attempted",
                  "PasLive": "Live-ball passes",
                  "PasDead": "Dead-ball passes",
                  "PasFK": "Passes attempted from free kicks",
                  "TB": "Completed pass sent between back defenders into open space",
                  "Sw": "Passes that travel more than 40 yards of the width of the pitch",
                  "PasCrs": "Crosses",
                  "TI": "Throw-Ins taken",
                  "CK": "Corner kicks",
                  "CkIn": "Inswinging corner kicks",
                  "CkOut": "Outswinging corner kicks",
                  "CkStr": "Straight corner kicks",
                  "PasCmp": "Passes completed",
                  "PasOff": "Offsides",
                  "PasBlocks": "Blocked by the opponent who was standing it the path",
                  "SCA": "Shot-creating actions",
                  "ScaPassLive": "Completed live-ball passes that lead to a shot attempt",
                  "ScaPassDead": "Completed dead-ball passes that lead to a shot attempt",
                  "ScaDrib": "Successful dribbles that lead to a shot attempt",
                  "ScaSh": "Shots that lead to another shot attempt",
                  "ScaFld": "Fouls drawn that lead to a shot attempt",
                  "ScaDef": "Defensive actions that lead to a shot attempt",
                  "GCA": "Goal-creating actions",
                  "GcaPassLive": "Completed live-ball passes that lead to a goal",
                  "GcaPassDead": "Completed dead-ball passes that lead to a goal",
                  "GcaDrib": "Successful dribbles that lead to a goal",
                  "GcaSh": "Shots that lead to another goal-scoring shot",
                  "GcaFld": "Fouls drawn that lead to a goal",
                  "GcaDef": "Defensive actions that lead to a goal",
                  "Tkl": "Number of players tackled",
                  "TklWon": "Tackles in which the tackler's team won possession of the ball",
                  "TklDef3rd": "Tackles in defensive 1/3",
                  "TklMid3rd": "Tackles in middle 1/3",
                  "TklAtt3rd": "Tackles in attacking 1/3",
                  "TklDri": "Number of dribblers tackled",
                  "TklDriAtt": "Number of times dribbled past plus number of tackles",
                  "TklDri%": "Percentage of dribblers tackled",
                  "TklDriPast": "Number of times dribbled past by an opposing player",
                  "Blocks": 'Number of times blocking the ball by standing in its path',
            'BlkSh': 'Number of times blocking a shot by standing in its path',
            'PKatt': 'Penalty kicks attempted',
            'PasTotCmp': 'Passes completed',
            'PasTotAtt': 'Passes attempted',
            'PasTotCmp%': 'Pass completion percentage',
            'PasTotDist': 'Total distance, in yards, that completed passes have traveled in any direction',
            'PasTotPrgDist': 'Total distance, in yards, that completed passes have traveled towards the opponent\'s goal',
            'PasShoCmp': 'Passes completed (Passes between 5 and 15 yards)',
            'PasShoAtt': 'Passes attempted (Passes between 5 and 15 yards)',
            'PasShoCmp%': 'Pass completion percentage (Passes between 5 and 15 yards)',
            'PasMedCmp': 'Passes completed (Passes between 15 and 30 yards)',
            'PasMedAtt': 'Passes attempted (Passes between 15 and 30 yards)',
            'PasMedCmp%': 'Pass completion percentage (Passes between 15 and 30 yards)',
            'PasLonCmp': 'Passes completed (Passes longer than 30 yards)',
            'PasLonAtt': 'Passes attempted (Passes longer than 30 yards)',
            'PasLonCmp%': 'Pass completion percentage (Passes longer than 30 yards)',
            'Assists': 'Assists',
            'PasAss': 'Passes that directly lead to a shot (assisted shots)',
            'Pas3rd': 'Completed passes that enter the 1/3 of the pitch closest to the goal',
            "PPA": "Completed passes into the 18-yard box",
            "CrsPA": "Completed crosses into the 18-yard box",
            "BlkPass": "Number of times blocking a pass by standing in its path",
                  "Int": "Interceptions",
                  "Tkl+Int": "Number of players tackled plus number of interceptions",
                  "Clr": "Clearances",
                  "Err": "Mistakes leading to an opponent's shot",
                  "Touches": "Number of times a player touched the ball. Note: Receiving a pass, then dribbling, then sending a pass counts as one touch",
                  "TouDefPen": "Touches in defensive penalty area",
                  "TouDef3rd": "Touches in defensive 1/3",
                  "TouMid3rd": "Touches in middle 1/3",
                  "TouAtt3rd": "Touches in attacking 1/3",
                  "TouAttPen": "Touches in attacking penalty area",
                  "TouLive": "Live-ball touches. Does not include corner kicks, free kicks, throw-ins, kick-offs, goal kicks or penalty kicks.",
                  "ToAtt": "Number of attempts to take on defenders while dribbling",
                  "ToSuc": "Number of defenders taken on successfully, by dribbling past them",
                  "ToSuc%": "Percentage of take-ons Completed Successfully",
                  "ToTkl": "Number of times tackled by a defender during a take-on attempt",
                  "ToTkl%": "Percentage of time tackled by a defender during a take-on attempt",
                  "Carries": "Number of times the player controlled the ball with their feet",
                  "CarTotDist": "Total distance, in yards, a player moved the ball while controlling it with their feet, in any direction",
                  "CarPrgDist": "Total distance, in yards, a player moved the ball while controlling it with their feet towards the opponent's goal",
                  "CarProg": "Carries that move the ball towards the opponent's goal at least 5 yards, or any carry into the penalty area",
                  "Car3rd": "Carries that enter the 1/3 of the pitch closest to the goal",
                  "CPA": "Carries into the 18-yard box",
                  "CarMis": "Number of times a player failed when attempting to gain control of a ball",
                  "CarDis": "Number of times a player loses control of the ball after being tackled by an opposing player",
                  "Rec": "Number of times a player successfully received a pass",
            "RecProg":
            "Completed passes that move the ball towards the opponents goal at least 10 yards from its furthest point in the last six passes, or any completed pass into the penalty area Make this a dictionary",
            "CrdY": "Yellow cards",
                  "CrdR": "Red cards",
                  "2CrdY": "Second yellow card",
                  "Fls": "Fouls committed",
                  "Fld": "Fouls drawn",
                  "Off": "Offsides",
                  "Crs": "Crosses",
                  "TklW": "Tackles in which the tackler's team won possession of the ball",
                  "PKwon": "Penalty kicks won",
                  "PKcon": "Penalty kicks conceded",
                  "OG": "Own goals",
                  "Recov": "Number of loose balls recovered",
                  "AerWon": "Aerials won",
                  "AerLost": "Aerials lost",
                  "AerWon%": "Percentage of aerials won",
            }
# dump this dict into a json
with open('./data/acronyms.json', 'w') as fp:
    json.dump(acronyms, fp)


In [30]:
# stampa i valori distinti di comp
print(df['Pos'].unique())

# stampa la lista delle features
print(list(df.columns))

['GK' 'DF' 'FW' 'MF' 'DFFW' 'MFFW' 'FWMF' 'DFMF' 'MFDF' 'FWDF' 'GKMF']
['Rk', 'Player', 'Nation', 'Pos', 'Comp', 'Age', 'Born', 'MP', 'Starts', 'Min', '90s', 'Goals', 'Shots', 'SoT', 'SoT%', 'G/Sh', 'G/SoT', 'ShoDist', 'ShoFK', 'ShoPK', 'PKatt', 'PasTotCmp', 'PasTotAtt', 'PasTotCmp%', 'PasTotDist', 'PasTotPrgDist', 'PasShoCmp', 'PasShoAtt', 'PasShoCmp%', 'PasMedCmp', 'PasMedAtt', 'PasMedCmp%', 'PasLonCmp', 'PasLonAtt', 'PasLonCmp%', 'Assists', 'PasAss', 'Pas3rd', 'PPA', 'CrsPA', 'PasProg', 'PasAtt', 'PasLive', 'PasDead', 'PasFK', 'TB', 'PasPress', 'Sw', 'PasCrs', 'CK', 'CkIn', 'CkOut', 'CkStr', 'PasGround', 'PasLow', 'PasHigh', 'PaswLeft', 'PaswRight', 'PaswHead', 'TI', 'PaswOther', 'PasCmp', 'PasOff', 'PasOut', 'PasInt', 'PasBlocks', 'SCA', 'ScaPassLive', 'ScaPassDead', 'ScaDrib', 'ScaSh', 'ScaFld', 'ScaDef', 'GCA', 'GcaPassLive', 'GcaPassDead', 'GcaDrib', 'GcaSh', 'GcaFld', 'GcaDef', 'Tkl', 'TklWon', 'TklDef3rd', 'TklMid3rd', 'TklAtt3rd', 'TklDri', 'TklDriAtt', 'TklDri%', 'TklDriPast

In [31]:
# stampa le rows di comp con Serie A e mi selezioni solo la colonna goals e player
df.loc[df['Comp'] == 'Serie A', ['Shots', 'Player']]

,Shots,Player
2833,0.00,C. Tătărușanu
517,1.43,Samu Castillejo
2489,0.76,A. Romagnoli
2411,3.03,A. Rebić
1015,0.17,M. Gabbia
...,...,...
3125,0.00,Alessandro Zanoli
3126,0.00,Mattia Zanotti
3148,1.02,Nadir Zortea
3151,0.00,Petar Zovko


In [32]:
# remove null and duplicate values
# df.dropna(inplace=True)
df.drop_duplicates(inplace=True)
df.drop_duplicates(inplace=True, subset=['Player'])

print(df.describe().transpose())

          count         mean         std   min       25%      50%       75%  \
Rk       2735.0  1463.269835  844.530377   1.0   736.500  1464.00  2195.500   
Age      2734.0    26.041697    4.682423  16.0    22.000    26.00    29.000   
Born     2735.0  1994.728702   38.441572   0.0  1992.000  1996.00  1999.000   
MP       2735.0    19.019378   11.630245   1.0     8.000    20.00    29.000   
Starts   2735.0    13.919196   11.428016   0.0     3.000    12.00    23.000   
...         ...          ...         ...   ...       ...      ...       ...   
OG       2735.0     0.003488    0.020603   0.0     0.000     0.00     0.000   
Recov    2735.0     7.418282    3.656263   0.0     5.100     7.38     9.375   
AerWon   2735.0     1.650567    1.938749   0.0     0.520     1.21     2.245   
AerLost  2735.0     1.833093    2.174223   0.0     0.835     1.39     2.190   
AerWon%  2735.0    40.437989   24.329573   0.0    26.100    43.50    57.100   

            max  
Rk       2921.0  
Age        41.0

In [33]:
competitions = ["Serie A", "Premier League", "La Liga", "Bundesliga", "Ligue 1"]

forward_positions = ["FW", "FWMF", "FWDF"]
midfield_positions = ["MF", "MFDF", "MFFW"]
defender_positions = ["DF", "DFMF", "DFFW"]

common_features = ["Player", "Squad", "Comp", "MP", "Pos", "90s", "PlayerFaceUrl", "ClubLogoUrl"]

forward_features = ["Goals", "Shots", "SoT", "G/Sh", "G/SoT", "ShoDist", "GCA", "SCA", "Off", "PKwon", "ScaDrib", "Assists",
                    "ScaPassLive", "Car3rd", "ScaFld",  "Carries", "CarTotDist", "CarPrgDist", 'CPA', "CarMis", "CarDis"]  # "ToAtt", "ToSuc"

midfielder_features = ["Goals", "PasTotCmp", "PasTotCmp%", "PasTotDist", "PasTotPrgDist", "Assists", "PasAss", "Pas3rd", "Crs", "PasCmp",
                       "PasOff", "PasBlocks", "SCA", "ScaPassLive", "ScaPassDead", "ScaDrib", "ScaSh", "ScaFld", "GCA", "GcaPassLive",
                       "GcaPassDead", "GcaDrib", "GcaSh", "GcaFld", "Tkl", "TklWon", "TklDef3rd", "TklMid3rd", "TklAtt3rd", "TklDri",
                       "TklDriAtt", "TklDri%", "TklDriPast", "Blocks", "BlkSh", "Int", "Recov", "Carries", "CarTotDist", "CarPrgDist", "Fld"]

defender_features = ["PasTotCmp", "PasTotDist", "PasTotPrgDist", "Tkl", "TklWon", "TklDef3rd", "TklMid3rd", "TklAtt3rd", "TklDri", "TklDriAtt", "TklDriPast", "Blocks",
                     "BlkSh", "Int", "Tkl+Int", "Recov", "AerWon", "AerLost", "Carries", "CarTotDist", "CarPrgDist", "CrdY", "CrdR", "Fls", "Clr"]


positions = [forward_positions,midfield_positions,defender_positions]

In [34]:
# removing outliers, who never player more than 180 mins
import os
dataset_path = './data/dataset-preproc'

df = df[df['90s'] > 5]

if not os.path.exists(dataset_path):
        os.makedirs(dataset_path)

for position in positions:
    pos, features_set = None, None
    
    if position == forward_positions:
        pos = "attk"
        features_set = common_features + forward_features
    elif position == midfield_positions:
        pos = "cen"
        features_set = common_features + midfielder_features
    else:
        pos = "dif"
        features_set = common_features + defender_features

    df_refined_1 = df[df["Pos"] == position[0]][features_set]
    df_refined_2 = df[df["Pos"] == position[1]][features_set]
    df_refined_3 = df[df["Pos"] == position[2]][features_set]
    df_forward = pd.concat([df_refined_1, df_refined_2, df_refined_3], axis=0)
    df_forward.to_csv(f'{dataset_path}/{pos}.csv', index=False)
        